Goals: 
- Find out precision, recall, F1 for each tag class in predictions compared to test set on a token level, not entity level.
- Can also use this to compare the unmodified test set to the modified test set
- This ignores differences between B-{tag} and I-{tag}

In [1]:
#Load the the original unmodified test set
'''
import csv
with open('./epi_test_setV2-corrected.tsv','r') as f:
    reader = csv.reader(f, delimiter="\t")
    pred_tokens = [row[0] for row in reader if len(row)>1]
    f.seek(0)
    pred_tags = [row[1] for row in reader if len(row)>1]
f.close()
'''

'\nimport csv\nwith open(\'./epi_test_setV2-corrected.tsv\',\'r\') as f:\n    reader = csv.reader(f, delimiter="\t")\n    pred_tokens = [row[0] for row in reader if len(row)>1]\n    f.seek(0)\n    pred_tags = [row[1] for row in reader if len(row)>1]\nf.close()\n'

In [2]:
#Load the predicted data (final custom dataset)
import csv
with open('./resultsV3.2/test_predictions.txt','r') as f:
    reader = csv.reader(f, delimiter=" ")
    '''
    for row in reader:
        if len(row)>1:
            print(row[0],row[1])
        else:
            print('')
    '''
    pred_tokens = [row[0] for row in reader if len(row)>1]
    f.seek(0)
    pred_tags = [row[1] for row in reader if len(row)>1]
f.close()

In [3]:
#Load the predicted data (large dataset)
'''
import csv
with open('./resultsLargeV3.1/test_predictions.txt','r') as f:
    reader = csv.reader(f, delimiter=" ")
    pred_tokens = [row[0] for row in reader if len(row)>1]
    f.seek(0)
    pred_tags = [row[1] for row in reader if len(row)>1]
f.close()
'''

'\nimport csv\nwith open(\'./resultsLargeV3.1/test_predictions.txt\',\'r\') as f:\n    reader = csv.reader(f, delimiter=" ")\n    pred_tokens = [row[0] for row in reader if len(row)>1]\n    f.seek(0)\n    pred_tags = [row[1] for row in reader if len(row)>1]\nf.close()\n'

In [4]:
for x in range(10):
    print(pred_tokens[x],pred_tags[x])

Context O
Patients O
with O
pseudohypoparathyroidism O
type O
1b O
( O
PHP1b O
) O
show O


In [5]:
#Load the corrected (modified) test set data
mod_tokens, mod_tags= [],[]
with open('./datasets/Large_DatasetV2/test.txt','r') as f:
    reader = csv.reader(f, delimiter=" ", quoting=csv.QUOTE_NONE)
    mod_tokens = [row[0] for row in reader if len(row)>1]
    f.seek(0)
    mod_tags = [row[1] for row in reader if len(row)>1]
f.close()

In [6]:
for x in range(10):
    print(mod_tokens[x],mod_tags[x])

Context O
Patients O
with O
pseudohypoparathyroidism O
type O
1b O
( O
PHP1b O
) O
show O


In [7]:
print(len(pred_tokens),len(mod_tokens))
print(len(pred_tags),len(mod_tags))
#Some of the sentences were combined when fixing the test set so there are now different number of sentences and different sizes of sentences

13910 13910
13910 13910


In [8]:
def compare_tag(pred_tag,mod_tag):
    
    changes, loc_add, loc_rem, stat_add, stat_rem, dz_add, dz_rem, epi_add, epi_rem=0,0,0,0,0,0,0,0,0
    
    if pred_tag != mod_tag:
        #This will add everything, including ones that change from I-LOC -> B-LOC, the others won't
        changes+=1
        
        #False positives
        if 'LOC' in pred_tag and 'LOC' not in mod_tag:
            loc_rem+=1
        elif 'DIS' in pred_tag and 'DIS' not in mod_tag:
            dz_rem+=1
        elif 'STAT' in pred_tag and 'STAT' not in mod_tag:
            stat_rem+=1
        elif 'EPI' in pred_tag and 'EPI' not in mod_tag:
            epi_rem+=1
        
        #False negatives
        if 'LOC' in mod_tag and 'LOC' not in pred_tag:
            loc_add+=1
        elif 'DIS' in mod_tag and 'DIS' not in pred_tag:
            dz_add+=1
        elif 'STAT' in mod_tag and 'STAT' not in pred_tag:
            stat_add+=1
        elif 'EPI' in mod_tag and 'EPI' not in pred_tag:
            epi_add+=1
        return changes, loc_add, loc_rem, stat_add, stat_rem, dz_add, dz_rem, epi_add, epi_rem
    else:
        return changes, loc_add, loc_rem, stat_add, stat_rem, dz_add, dz_rem, epi_add, epi_rem

In [9]:
mismatch, loc_fn, loc_fp, stat_fn, stat_fp, dz_fn, dz_fp, epi_fn,epi_fp=0,0,0,0,0,0,0,0,0
mod_idx, pred_idx=0,0
skipped_annotations=0

while pred_idx<len(pred_tokens) and mod_idx<len(mod_tokens):
    if mod_tokens[mod_idx] != pred_tokens[pred_idx]:
        if mod_tokens[mod_idx+1] == pred_tokens[pred_idx]:
            mod_idx+=1
            #compare
            changes, loc_add, loc_rem, stat_add, stat_rem, dz_add, dz_rem, epi_add, epi_rem = compare_tag(
                pred_tags[pred_idx],mod_tags[mod_idx])
            mismatch+=changes
            loc_fn+=loc_add
            loc_fp+=loc_rem
            stat_fn+=stat_add
            stat_fp+=stat_rem
            dz_fn+=dz_add
            dz_fp+=dz_rem
            epi_fn+=epi_add
            epi_fp+=epi_rem
            
            pred_idx+=1
            mod_idx+=1
        elif mod_tokens[mod_idx] == pred_tokens[pred_idx+1]:
            pred_idx+=1
            #compare
            changes, loc_add, loc_rem, stat_add, stat_rem, dz_add, dz_rem, epi_add, epi_rem = compare_tag(
                pred_tags[pred_idx],mod_tags[mod_idx])
            mismatch+=changes
            loc_fn+=loc_add
            loc_fp+=loc_rem
            stat_fn+=stat_add
            stat_fp+=stat_rem
            dz_fn+=dz_add
            dz_fp+=dz_rem
            epi_fn+=epi_add
            epi_fp+=epi_rem
            
            pred_idx+=1
            mod_idx+=1
        
        else:
            print('uh oh-->','mod_idx:',mod_idx,', pred_idx:',pred_idx)
            print(mod_tokens[mod_idx],pred_tokens[pred_idx])
            skipped_annotations+=1
            pred_idx+=1
            mod_idx+=1
            #skip compare
    else:
        #compare
        changes, loc_add, loc_rem, stat_add, stat_rem, dz_add, dz_rem, epi_add, epi_rem = compare_tag(
            pred_tags[pred_idx],mod_tags[mod_idx])
        mismatch+=changes
        loc_fn+=loc_add
        loc_fp+=loc_rem
        stat_fn+=stat_add
        stat_fp+=stat_rem
        dz_fn+=dz_add
        dz_fp+=dz_rem
        epi_fn+=epi_add
        epi_fp+=epi_rem
        
        pred_idx+=1
        mod_idx+=1
            

print('mismatch: ',mismatch, 
      '\nloc_fn: ',loc_fn, 
      '\nloc_fp: ',loc_fp, 
      '\nstat_fn: ',stat_fn, 
      '\nstat_fp: ',stat_fp, 
      '\ndz_fn: ',dz_fn, 
      '\ndz_fp: ',dz_fp, 
      '\nepi_fn: ',epi_fn, 
      '\nepi_fp: ',epi_fp,
      '\nskipped annotations: ',int(skipped_annotations))

mismatch:  296 
loc_fn:  39 
loc_fp:  6 
stat_fn:  125 
stat_fp:  83 
dz_fn:  0 
dz_fp:  0 
epi_fn:  9 
epi_fp:  11 
skipped annotations:  0


In [10]:
def count_tags(tags):
    l,d,s,e = 0,0,0,0
    for tag in tags:
        if 'LOC' in tag:
            l+=1
        elif 'DIS' in tag:
            d+=1
        elif 'STAT' in tag:
            s+=1
        elif 'EPI' in tag:
            e+=1
    return l,d,s,e

In [11]:
def get_stats(tp,fp,fn):
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    F1=(2*precision*recall)/(precision+recall)
    return {'precision':precision,'recall':recall,'F1':F1}

In [12]:
loc_tp, dz_tp, stat_tp, epi_tp = count_tags(mod_tags)
#dz_stats = get_stats(dz_tp,dz_fp,dz_fn)
loc_stats = get_stats(loc_tp,loc_fp,loc_fn)
stat_stats = get_stats(stat_tp,stat_fp,stat_fn)
epi_stats = get_stats(epi_tp,epi_fp,epi_fn)
overall_stats = get_stats(loc_tp+dz_tp+stat_tp+epi_tp, loc_fp+dz_fp+stat_fp+epi_fp, dz_fn+loc_fn+stat_fn+epi_fn)


for k,v in overall_stats.items():
    print('token-level overall',k,v)
print('')
#for k,v in dz_stats.items():
#    print('disease classification',k,v)
#print('')
for k,v in loc_stats.items():
    print('location classification',k,v)
print('')
for k,v in stat_stats.items():
    print('statistic identification',k,v)
print('')
for k,v in epi_stats.items():
    print('epidemiology word classification',k,v)
print('')
print('overall accuracy',1-(mismatch)/(len(mod_tags)-skipped_annotations))

token-level overall precision 0.8109640831758034
token-level overall recall 0.7126245847176079
token-level overall F1 0.7586206896551723

location classification precision 0.9491525423728814
location classification recall 0.7417218543046358
location classification F1 0.8327137546468403

statistic identification precision 0.7242524916943521
statistic identification recall 0.6355685131195336
statistic identification F1 0.6770186335403726

epidemiology word classification precision 0.9
epidemiology word classification recall 0.9166666666666666
epidemiology word classification F1 0.908256880733945

overall accuracy 0.9787203450754852
